In [ ]:
# %% [markdown]
# # 2. Schools Data: INEP & Censo Escolar
#
# **The Problem:** School data is split. One file has the address (lat/lon); another massive file has the metrics (students, staff, labs).
#
# **The Solution:** AtlasBR performs an automatic **Data Fusion**, giving you a single GeoDataFrame ready for mapping.
#
# **What you will learn:**
# 1.  **Rich Points:** Loading schools with attached metrics.
# 2.  **Filtering:** Analyzing the Public vs. Private divide.

# %%
import os
import atlasbr
from atlasbr.app.inep import load_schools

atlasbr.configure_logging()
MUNICIPALITY = "Florianópolis, SC"

# %% [markdown]
# ## 1. The Fusion Load
# We request schools for 2023. Under the hood, AtlasBR is joining `br_bd_diretorios_brasil.escola` with `br_inep_censo_escolar`.

# %%
gdf_schools = load_schools(
    places=[MUNICIPALITY],
    year=2023,
    as_gdf=True,  # Return as Points (False returns raw Table)
    gcp_billing=os.getenv("GOOGLE_CLOUD_PROJECT") # Required for BD
)

print(f"🎓 Found {len(gdf_schools)} schools.")
display(gdf_schools.head(3))

# %% [markdown]
# ## 2. Public vs. Private Network
# AtlasBR standardizes the `rede` column (State, Municipal, Federal, Private) to simplify analysis.

# %%
# Let's count the split
print(gdf_schools["rede"].value_counts())

# %% [markdown]
# ## 3. Interactive Map: The Education Landscape
# Zoom in to see where schools are clustered. Click on a point to see the student count.
# * **Blue:** Public Schools
# * **Red:** Private Schools

# %%
# Simple color mapping helper
def color_school(val):
    return "blue" if val == "Publica" else "red"

gdf_schools["color"] = gdf_schools["rede"].apply(color_school)

gdf_schools.explore(
    column="rede",
    cmap="Set1", # Categorical Colormap
    marker_kwds={"radius": 6},
    tooltip=[
        "id_escola", 
        "rede", 
        "quantidade_matricula_fundamental", 
        "quantidade_docente_educacao_basica"
    ],
    tiles="CartoDB Positron"
)